In [1]:
import EnerFrame as ef
import pandas as pd
import os
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

KeyboardInterrupt: 

In [ ]:
cwd = os.getcwd()
inputDataPath = os.path.join(cwd,"InputData")

In [ ]:
CO2modeling=True
biogasModeling=True
H2modeling=True
transmissionModeling=True
storageModeling=True
RESmodeling=True
RORmodeling=True
PHSmodeling=True
maxCapCCGTCH4modeling=False

locations = {'cluster_0','cluster_7'}
commodities = {'electricity','methane'}
if biogasModeling:
    commodities.add('biogas')
if CO2modeling:
    commodities.add('CO2')
if H2modeling:
    commodities.add('hydrogen')
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [ ]:
commodities

In [ ]:
CO2_reductionTarget = 0.95
CO2reduction = 366*(1-CO2_reductionTarget)*6/8

In [ ]:
CO2reduction

In [ ]:
if not CO2modeling:
    esM = ef.EnergySystemStructure(regions=locations, carriers=commodities, 
                                   powerUnit='GW', energyUnit='GWh',
                                   costUnit='1e9 Euro', lengthUnit='km')
else:
    esM = ef.EnergySystemStructure(regions=locations, carriers=commodities, 
                                   powerUnit='GW', energyUnit='GWh',
                                   costUnit='1e9 Euro', lengthUnit='km', carrierLimitation={'CO2':CO2reduction})
    

In [ ]:
if RESmodeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxCapacityOnshore_GW_el.xlsx'),
                              index_col=0, squeeze=True)
    operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxOperationRateOnshore_el.xlsx'))

    esM.add(ef.GenerationTechnology(esS=esM, name='Wind (onshore)', carrier='electricity', 
                      maxUtilizationRate=operationRateMax, maxCapacity=capacityMax,
                      capexCap=1.1, opexCap=1.1*0.02, interest=0.08,
                      economicLifetime=20, powerUnit='GW', energyUnit='GWh', costUnit='1e9 Euro'))

In [ ]:
if RESmodeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxCapacityOffshore_GW_el.xlsx'),
                              index_col=0, squeeze=True)
    operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxOperationRateOffshore_el.xlsx'))
    
    esM.add(ef.GenerationTechnology(esS=esM, name='Wind (offshore)', carrier='electricity', 
                      maxUtilizationRate=operationRateMax, maxCapacity=capacityMax,
                      capexCap=2.3, opexCap=2.3*0.02, interest=0.08,
                      economicLifetime=20, powerUnit='GW', energyUnit='GWh', costUnit='1e9 Euro'))    

In [ ]:
if RESmodeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','PV','maxCapacityPV_GW_el.xlsx'),
                              index_col=0, squeeze=True)
    operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','PV','maxOperationRatePV_el.xlsx'))
    
    esM.add(ef.GenerationTechnology(esS=esM, name='PV', carrier='electricity', 
                      maxUtilizationRate=operationRateMax, maxCapacity=capacityMax,
                      capexCap=0.65, opexCap=0.65*0.02, interest=0.08,
                      economicLifetime=25, powerUnit='GW', energyUnit='GWh', costUnit='1e9 Euro'))       

In [ ]:
if RESmodeling and RORmodeling:
    capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower','fixCapacityROR_GW_el.xlsx'),
                              index_col=0, squeeze=True)
    operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower',
                                                'fixOperationRateROR_GW_el.xlsx'))
    
    esM.add(ef.GenerationTechnology(esS=esM, name='Existing run-of-river plants', carrier='electricity', 
                      maxUtilizationRate=operationRateFix, fixCapacity=capacityFix, isDispatchable=False,
                      capexCap=0, opexCap=0.208, interest=0.08,
                      economicLifetime=60, powerUnit='GW', energyUnit='GWh', costUnit='1e9 Euro'))      

In [ ]:
esM.add(ef.PurchaseOption(esS=esM, name='Natural gas purchase', carrier='methane',
                  energyUnit='GWh', costUnit='1e9 Euro', cost=0.0331*1e-3))

In [ ]:
if biogasModeling:
    operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Biogas',
                                                'biogasPotential_GWh_biogas.xlsx'))
    
    esM.add(ef.PurchaseOption(esS=esM, name='Biogas purchase', carrier='biogas',
                      energyUnit='GWh', costUnit='1e9 Euro', cost=0.05409*1e-3))    

In [ ]:
if CO2modeling and biogasModeling:
    esM.add(ef.PurchaseOption(esS=esM, name='CO2 from enviroment', carrier='CO2',
                      energyUnit='GWh', costUnit='1e9 Euro', cost=0, carrierLimitationKey='CO2'))   

In [ ]:
if CO2modeling and biogasModeling:
    esM.add(ef.ConversionTechnology(esS=esM, name='Biogas to methane',
                          conversionFactors={'biogas':-1, 'methane':1, 'CO2':-201*1e-6*0.625, 'electricity':0, 'hydrogen':0},
                          energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro', economicLifetime=10, interest=0.08))

In [ ]:
if maxCapCCGTCH4modeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','NaturalGasPlants',
                                           'existingCombinedCycleGasTurbinePlantsCapacity_GW_el.xlsx'),
                              index_col=0, squeeze=True)
else:
    capacityMax=pd.Series()
    
if CO2modeling and H2modeling and biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625, 'biogas':0, 'hydrogen':0}
elif CO2modeling and H2modeling and not biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625, 'hydrogen':0}
elif CO2modeling and not H2modeling and biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625, 'biogas':0}
elif CO2modeling and not H2modeling and not biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625}    
elif not CO2modeling and H2modeling and biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'biogas':0, 'hydrogen':0}    
elif not CO2modeling and H2modeling and not biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'hydrogen':0}
elif not CO2modeling and not H2modeling and biogasModeling:
    cF = {'electricity':1, 'methane':-1/0.625, 'biogas':0}
else:
    cF = {'electricity':1, 'methane':-1/0.625}    

esM.add(ef.ConversionTechnology(esS=esM, name='Existing CCGT plants (methane)',
                      conversionFactors=cF,
                      capexCap=0, opexCap=0.021, energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                      economicLifetime=33, interest=0.08, maxCapacity=capacityMax))                                        

In [ ]:
if H2modeling and not CO2modeling:
    cF = {'electricity':1, 'biogas':-1/0.635, 'methane':0, 'hydrogen':0}
elif H2modeling and CO2modeling:
    cF = {'electricity':1, 'biogas':-1/0.635, 'methane':0, 'hydrogen':0,  'CO2':0}    
else:
    cF = {'electricity':1, 'biogas':-1/0.635, 'methane':0}
    
if biogasModeling:
    esM.add(ef.ConversionTechnology(esS=esM, name='New CCGT plants (biogas)',
                          conversionFactors=cF,
                          capexCap=0.7, opexCap=0.021, energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                          economicLifetime=33, interest=0.08))   

In [ ]:
if biogasModeling and CO2modeling:
    cF = {'electricity':1, 'biogas':0, 'methane':0, 'hydrogen':-1/0.6, 'CO2':0}
elif not biogasModeling and CO2modeling:
    cF = {'electricity':1, 'methane':0, 'hydrogen':-1/0.6, 'CO2':0}
else:
    cF = {'electricity':1, 'methane':0, 'hydrogen':-1/0.6}
    
if H2modeling:
    esM.add(ef.ConversionTechnology(esS=esM, name='New CCGT plants (hydrogen)',
                          conversionFactors=cF,
                          capexCap=0.927, opexCap=0.021, energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                          economicLifetime=33, interest=0.08))   

In [ ]:
if biogasModeling and CO2modeling:
    cF = {'electricity':-1, 'biogas':0, 'methane':0, 'hydrogen':0.7, 'CO2':0}
elif not biogasModeling and CO2modeling:
    cF = {'electricity':-1, 'methane':0, 'hydrogen':0.7, 'CO2':0}
else:
    cF = {'electricity':-1, 'methane':0, 'hydrogen':0.7}
    
if H2modeling:
    esM.add(ef.ConversionTechnology(esS=esM, name='Electrolyzers',
                          conversionFactors=cF,
                          capexCap=0.5, opexCap=0.5*0.025, energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                          economicLifetime=10, interest=0.08))   

In [ ]:
if storageModeling:
    esM.add(ef.StorageTechnology(esS=esM, name='Li-ion batteries', carrier='electricity',
                                 capexCap=0.151, opexCap=0.002, #cyclicLifetime=10000, 
                                 injectionEfficiency=0.95, deliveryEfficiency=0.95,
                                 injectability=1, deliverability=1, preciseInventoryLimitation=True,
                                 selfDischarge=1-(1-0.03)**(1/(30*24)), 
                                 energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                                 economicLifetime=22, interest=0.08))   

In [ ]:
if H2modeling and storageModeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','GeologicalStorage',
                                           'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                              index_col=0, squeeze=True)*3/10

    esM.add(ef.StorageTechnology(esS=esM, name='Salt caverns (hydrogen)', carrier='hydrogen',
                                 capexCap=0.00011, opexCap=0.00057, #cyclicLifetime=10000, 
                                 multipleUsageID='Existing salt caverns', minSOC=0.33, maxSOC=1,
                                 injectability=1/470.37, deliverability=1/470.37,
                                 energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                                 economicLifetime=30, interest=0.08, maxCapacity=capacityMax))  

In [ ]:
if storageModeling and biogasModeling:
    capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','GeologicalStorage',
                                           'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                              index_col=0, squeeze=True)

    esM.add(ef.StorageTechnology(esS=esM, name='Salt caverns (methane)', carrier='methane',
                                 capexCap=0.00004, opexCap=0.00001, #cyclicLifetime=10000, 
                                 multipleUsageID='Existing salt caverns', minSOC=0.33, maxSOC=1,
                                 injectability=1/470.37, deliverability=1/470.37,
                                 energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                                 economicLifetime=30, interest=0.08, maxCapacity=capacityMax))      

In [ ]:
if storageModeling and PHSmodeling:
    capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower',
                                           'fixCapacityPHS_storage_GWh_energyPHS.xlsx'),
                              index_col=0, squeeze=True)

    esM.add(ef.StorageTechnology(esS=esM, name='Pumped hydro storage', carrier='electricity',
                                 injectionEfficiency=0.88, deliveryEfficiency=0.88,
                                 selfDischarge=1-(1-0.00375)**(1/(30*24)),
                                 capexCap=0, opexCap=0.000153, 
                                 injectability=0.16, deliverability=0.12, 
                                 energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro',
                                 economicLifetime=30, interest=0.08, fixCapacity=capacityFix))      

In [ ]:
if transmissionModeling:
    capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                           'ACcableExistingCapacity_GW_el.xlsx'),
                              index_col=0, header=0)
    eligibility= capacityFix.copy()
    eligibility[eligibility>0] =1

    esM.add(ef.TransmissionTechnology(esS=esM, name='AC cables', carrier='electricity', eligibility=eligibility,
                                      fixCapacity=capacityFix, economicLifetime=40, interest=0.08, lengths=capacityFix,
                                      energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro', lengthUnit='km'))

In [ ]:
if transmissionModeling:
    capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                           'DCcableExistingCapacity_GW_el.xlsx'),
                              index_col=0, header=0)
    eligibility= capacityFix.copy()
    eligibility[eligibility>0] =1
    
    distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                           'DCcableLength_km.xlsx'),
                              index_col=0, header=0)

    esM.add(ef.TransmissionTechnology(esS=esM, name='DC cables', carrier='electricity', eligibility=eligibility,
                                      losses=0.00001,
                                      fixCapacity=capacityFix, economicLifetime=40, interest=0.08, lengths=distances,
                                      energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro', lengthUnit='km'))    

In [ ]:
if transmissionModeling:
    eligibility=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines',
                                           'pipelineIncidence.xlsx'), index_col=0, header=0)
    distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                              index_col=0, header=0)

    esM.add(ef.TransmissionTechnology(esS=esM, name='Pipelines (methane)', carrier='methane',
                                      eligibility=eligibility, capexCap=0.000037,
                                      economicLifetime=40, interest=0.08, lengths=distances,
                                      energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro', lengthUnit='km'))      

In [ ]:
if H2modeling and transmissionModeling:
    eligibility=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                              index_col=0, header=0)
    distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                              index_col=0, header=0)
    
    esM.add(ef.TransmissionTechnology(esS=esM, name='Pipelines (hydrogen)', carrier='hydrogen',
                                      eligibility=eligibility, capexCap=0.000177,
                                      economicLifetime=40, interest=0.08, lengths=distances,
                                      energyUnit='GWh', powerUnit='GW', costUnit='1e9 Euro', lengthUnit='km'))       

In [ ]:
operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Demands',
                                            'electricityDemand_GWh_el.xlsx'))

esM.add(ef.DemandData(esS=esM, name='Electricity demand', carrier='electricity',
                data=operationRateFix, energyUnit='GWh'))

In [ ]:
if H2modeling:
    FCEV_penetration=0.5
    operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Demands',
                                                'hydrogenDemand_GWh_hydrogen.xlsx'))*FCEV_penetration

    esM.add(ef.DemandData(esS=esM, name='Hydrogen demand', carrier='hydrogen',
                    data=operationRateFix, energyUnit='GWh'))    

In [ ]:
if CO2modeling:
    esM.add(ef.SaleOption(esS=esM, name='CO2 to enviroment', carrier='CO2',
                      energyUnit='GWh', costUnit='1e9 Euro', cost=0, carrierLimitationKey='CO2'))   

In [ ]:
periods = []
TAC = []
cap_conv = []
cap_sosi = []
cap_stor = []

l1=range(4,30,4)
l2=range(30,150,30)
l3=range(150,360,70)
l=[]
l.extend(l1), l.extend(l2), l.extend(l3), l.extend([365])
l = [7]

for p in l:
    esM.cluster(numberOfTypicalPeriods=p)
    pyM, results, _, _ = ef.optimize(esM, timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-6 cuts=3 method=2')
    TAC.append(pyM.Obj()), periods.append(p)
    fig, ax1 = plt.subplots(1,1, figsize=(12,4))
    ax1.plot(periods, TAC)       
    plt.show()

#pyM, results, _, _ = ef.optimize(esM, timeSeriesAggregation=False, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-6 cuts=3 method=2')
#TAC.append(pyM.Obj()), periods.append(366)
#fig, ax1 = plt.subplots(1,1, figsize=(12,4))
#ax1.plot(periods, TAC)       
#plt.show()

In [ ]:
6.59491872599
6.59487771323

In [ ]:
for i in pyM.timeSet:
    print(i)

In [ ]:
ef.outputManager.getOptimizationSummary(esM, results, True).sum(axis=1)

In [ ]:
CO2reduction